In [1]:
import excelify as el
import polars as pl
import numpy as np

### Scratch

In [2]:
df = el.ExcelFrame({
    "x": [1, 2, 3]
})

In [3]:
df = df.with_columns(el.Col("x").alias("y"))

In [4]:
df.evaluate()

x,y
1,1
2,2
3,3


In [5]:
df = df.with_columns(el.Col("y").prev(1).alias("z"))

In [5]:
df = df.with_column((el.Col("x") / el.Col("y")).alias("z"))

In [5]:
df = df.with_column(el.MultCol(el.Col("x"), el.Col("y")).alias("z"))

In [9]:
df.columns

['x', 'y', 'z']

In [9]:
df["z"]

In [6]:
df.evaluate()

x,y,z
1,1,0
2,2,1
3,3,2


In [6]:
df.to_excel("example.xlsx")

In [4]:
cell = el.ExcelFrame.of_excel("example.xlsx")
cell

In [6]:
cell(2)

2

In [4]:
func.inputs

OrderedDict([('A2', <Ranges>(A2)), ('B3', <Ranges>(B3))])

In [9]:
df

,x,0,0.1,0.2
0,y,0,0,0
1,z,0,0,0


In [3]:
df["x"]

[Const(1), Const(2), Const(3)]

## DCF using Python

### Step 1: Raw Input Data

In [1]:
import excelify as el
import numpy as np

In [2]:
tax_rates = [6858. / 20564., 4915. / 20116., 4281. / 11460.]
effective_tax_rate = sum(tax_rates) / len(tax_rates)

In [3]:
df = el.ExcelFrame(
    {
        "Year": ["FY19", "FY20", "FY21"],
        "Retail Square Foot": [1129, 1129, 1121],
        "Net Sales": [510_329, 519_926, 555_233],
        "Membership & Other Income": [4076, 4038, 3918],
        "Operating Income (EBIT)": [21_957, 20_568, 22_548],
        "Capital Expenditures": [-10_344, -10_705, -10_604],
        "Depreciation & Amortization": [10_678, 10_987, 11_152],
    }
)
fy18_retail_square_foot = 1158

In [4]:
df

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization
FY19,1129,510329,4076,21957,-10344,10678
FY20,1129,519926,4038,20568,-10705,10987
FY21,1121,555233,3918,22548,-10604,11152


In [5]:
[quarter, retail_square_foot, net_sales, other_income, ebit, capex, d_and_a] = [
    el.col(c) for c in df.columns
]

In [8]:
df = df.with_columns(
    (net_sales / retail_square_foot).alias("Sales per Square Foot"),
    ((net_sales - ebit) / retail_square_foot).alias("COGS and OpEx per Square Foot"),
    (-capex / retail_square_foot.prev(1)).alias("Maintenance CapEx per Square Foot"),
    (d_and_a / retail_square_foot).alias("D&A per Square Foot"),
    (net_sales + other_income).alias("Total Revenue"),
    (-ebit * effective_tax_rate).alias("(-) Taxes, Excluding Effect of Interest:"),
)
df = df.with_columns(
    (el.col("(-) Taxes, Excluding Effect of Interest:") + ebit).alias(
        "Net Operating Profit After Tax (NOPAT)"
    ),
)
df["Maintenance CapEx per Square Foot"][0] = -df["Capital Expenditures"][0] / el.Constant(fy18_retail_square_foot)

In [7]:
df.evaluate()

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization,Sales per Square Foot,COGS and OpEx per Square Foot,Maintenance CapEx per Square Foot,D&A per Square Foot,Total Revenue,"(-) Taxes, Excluding Effect of Interest:",Net Operating Profit After Tax (NOPAT)
FY19,1129,510329,4076,21957,-10344,10678,452.0186005314438,432.5704162976085,8.932642487046632,9.45792736935341,514405,-6963.212487758894,14993.787512241106
FY20,1129,519926,4038,20568,-10705,10987,460.51904340124,442.30115146147034,9.481842338352525,9.731620903454385,523964,-6522.719608699956,14045.280391300044
FY21,1121,555233,3918,22548,-10604,11152,495.30151650312223,475.18733273862625,9.392382639503985,9.948260481712756,559151,-7150.6360237731715,15397.363976226829


### Step 2: Projections

In [12]:
projected_years = [f"FY{22 + i}" for i in range(10)]
retail_square_foot_gr = [0.02] * 2 + [0.015] * 2 + [0.01] * 2 + [0.005] * 4
sales_per_sqft_gr = [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2
cogs_and_opex_per_sqft_gr = sales_per_sqft_gr
maintenance_capex_per_sqft_gr = sales_per_sqft_gr
d_and_a_per_sqft_gr = [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2 + [0.008] * 2
initial_growth_capex_per_sqft = 150.
growth_capex_per_sqft_gr = [np.nan] + [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01]
membersip_and_other_income_gr = [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2

In [13]:
manual_inputs = el.ExcelFrame(
    {
        "Year": projected_years,
        "Retail Square Foot Growth Rate": retail_square_foot_gr,
        "Sales per Square Foot Growth Rate": sales_per_sqft_gr,
        "COGS and OPEX per Square Foot Growth Rate": cogs_and_opex_per_sqft_gr,
        "Maintenance CapEx per Square Foot Growth Rate": maintenance_capex_per_sqft_gr,
        "D&A per Square Foot Growth Rate": d_and_a_per_sqft_gr,
        "Growth CapEx per New Square Foot Growth Rate": growth_capex_per_sqft_gr,
        "Membership and Other Income Growth Rate": membersip_and_other_income_gr,
    }
)
manual_inputs

Year,Retail Square Foot Growth Rate,Sales per Square Foot Growth Rate,COGS and OPEX per Square Foot Growth Rate,Maintenance CapEx per Square Foot Growth Rate,D&A per Square Foot Growth Rate,Growth CapEx per New Square Foot Growth Rate,Membership and Other Income Growth Rate
FY22,0.02,0.03,0.03,0.03,0.025,nan,0.03
FY23,0.02,0.03,0.03,0.03,0.025,0.03,0.03
FY24,0.015,0.025,0.025,0.025,0.02,0.03,0.025
FY25,0.015,0.025,0.025,0.025,0.02,0.025,0.025
FY26,0.01,0.02,0.02,0.02,0.015,0.025,0.02
FY27,0.01,0.02,0.02,0.02,0.015,0.02,0.02
FY28,0.005,0.015,0.015,0.015,0.01,0.02,0.015
FY29,0.005,0.015,0.015,0.015,0.01,0.015,0.015
FY30,0.005,0.01,0.01,0.01,0.008,0.015,0.01
FY31,0.005,0.01,0.01,0.01,0.008,0.01,0.01


In [10]:
lf = df.with_columns(
    (net_sales / retail_square_foot).alias("Sales per Square Foot"),
    ((net_sales - ebit) / retail_square_foot).alias("COGS and OpEx per Square Foot"),
    (-capex / retail_square_foot.prev(1).fill_null(fy18_retail_square_foot)).alias(
        "Maintenance CapEx per Square Foot"
    ),
    (d_and_a / retail_square_foot).alias("D&A per Square Foot"),
    (net_sales + other_income).alias("Total Revenue"),
    (-ebit * effective_tax_rate).alias("(-) Taxes, Excluding Effect of Interest:"),
)
lf = lf.with_columns(
    (pl.col("(-) Taxes, Excluding Effect of Interest:") + ebit).alias(
        "Net Operating Profit After Tax (NOPAT)"
    ),
)
historical_df = lf.collect()

AttributeError: 'Col' object has no attribute 'fill_null'

In [4]:
df = pl.DataFrame(
    {
        "Year": ["FY19", "FY20", "FY21"],
        "Retail Square Foot": [1129, 1129, 1121],
        "Net Sales": [510_329, 519_926, 555_233],
        "Membership & Other Income": [4076, 4038, 3918],
        "Operating Income (EBIT)": [21_957, 20_568, 22_548],
        "Capital Expenditures": [-10_344, -10_705, -10_604],
        "Depreciation & Amortization": [10_678, 10_987, 11_152],
    }
)
fy18_retail_square_foot = 1158

In [7]:
df

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization
str,i64,i64,i64,i64,i64,i64
"""FY19""",1129,510329,4076,21957,-10344,10678
"""FY20""",1129,519926,4038,20568,-10705,10987
"""FY21""",1121,555233,3918,22548,-10604,11152


### Design

Programmable Excel - Maybe a better VBA

Why is this different from dataframes?
Dataframe is not a DAG computation graph, but Excel is. But Excel looks very similar to Dataframe in a way that it's tabular.

So I'd like to build a python library that makes it easier to define an excel-like table, like:

```python

df = el.excelFrame({"x": [1, 2, 3]})
df = df.with_columns(
    # It'll define "y" as "x" column cell * 2 for each row.
    (el.col("x") * 2).alias("y"),
    # This is a common financial excel pattern (e.g. previous earnings * expected growth rate).
    (el.col("y").prev(1) * el.col("x")).alias("z"),
    # Empty column.
    (el.empty()).alias("empty_col")
)

df["empty_col"][2] = df["x"][0] + df["y"][0]
df["y"][:3].clear()

df.write_excel()

```

In [43]:
[quarter, retail_square_foot, net_sales, other_income, ebit, capex, d_and_a] = [
    pl.col(c) for c in df.columns
]

### Derive Other Columns

In [44]:
lf = df.lazy().with_columns(
    (net_sales / retail_square_foot).alias("Sales per Square Foot"),
    ((net_sales - ebit) / retail_square_foot).alias("COGS and OpEx per Square Foot"),
    (-capex / retail_square_foot.shift(1).fill_null(fy18_retail_square_foot)).alias(
        "Maintenance CapEx per Square Foot"
    ),
    (d_and_a / retail_square_foot).alias("D&A per Square Foot"),
    (net_sales + other_income).alias("Total Revenue"),
    (-ebit * effective_tax_rate).alias("(-) Taxes, Excluding Effect of Interest:"),
)
lf = lf.with_columns(
    (pl.col("(-) Taxes, Excluding Effect of Interest:") + ebit).alias(
        "Net Operating Profit After Tax (NOPAT)"
    ),
)
historical_df = lf.collect()

### Projections

In [45]:
projected_years = [f"FY{22 + i}" for i in range(10)]
retail_square_foot_gr = [0.02] * 2 + [0.015] * 2 + [0.01] * 2 + [0.005] * 4
sales_per_sqft_gr = [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2
cogs_and_opex_per_sqft_gr = sales_per_sqft_gr
maintenance_capex_per_sqft_gr = sales_per_sqft_gr
d_and_a_per_sqft_gr = [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2 + [0.008] * 2
initial_growth_capex_per_sqft = 150.
growth_capex_per_sqft_gr = [np.nan] + [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01]
membersip_and_other_income_gr = [0.03] * 2 + [0.025] * 2 + [0.02] * 2 + [0.015] * 2 + [0.01] * 2

In [46]:
manual_inputs = pl.DataFrame(
    {
        "Year": projected_years,
        "Retail Square Foot Growth Rate": retail_square_foot_gr,
        "Sales per Square Foot Growth Rate": sales_per_sqft_gr,
        "COGS and OPEX per Square Foot Growth Rate": cogs_and_opex_per_sqft_gr,
        "Maintenance CapEx per Square Foot Growth Rate": maintenance_capex_per_sqft_gr,
        "D&A per Square Foot Growth Rate": d_and_a_per_sqft_gr,
        "Growth CapEx per New Square Foot Growth Rate": growth_capex_per_sqft_gr,
        "Membership and Other Income Growth Rate": membersip_and_other_income_gr,
    }
)
manual_inputs

Year,Retail Square Foot Growth Rate,Sales per Square Foot Growth Rate,COGS and OPEX per Square Foot Growth Rate,Maintenance CapEx per Square Foot Growth Rate,D&A per Square Foot Growth Rate,Growth CapEx per New Square Foot Growth Rate,Membership and Other Income Growth Rate
str,f64,f64,f64,f64,f64,f64,f64
"""FY22""",0.02,0.03,0.03,0.03,0.025,NaN,0.03
"""FY23""",0.02,0.03,0.03,0.03,0.025,0.03,0.03
"""FY24""",0.015,0.025,0.025,0.025,0.02,0.03,0.025
"""FY25""",0.015,0.025,0.025,0.025,0.02,0.025,0.025
"""FY26""",0.01,0.02,0.02,0.02,0.015,0.025,0.02
"""FY27""",0.01,0.02,0.02,0.02,0.015,0.02,0.02
"""FY28""",0.005,0.015,0.015,0.015,0.01,0.02,0.015
"""FY29""",0.005,0.015,0.015,0.015,0.01,0.015,0.015
"""FY30""",0.005,0.01,0.01,0.01,0.008,0.015,0.01


In [47]:
joined_df = (
    historical_df.join(manual_inputs, on="Year", how="full")
    .with_columns(pl.coalesce(["Year", "Year_right"]).alias("Year"))
    .drop("Year_right")
    .sort("Year")
)
joined_df

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization,Sales per Square Foot,COGS and OpEx per Square Foot,Maintenance CapEx per Square Foot,D&A per Square Foot,Total Revenue,"(-) Taxes, Excluding Effect of Interest:",Net Operating Profit After Tax (NOPAT),Retail Square Foot Growth Rate,Sales per Square Foot Growth Rate,COGS and OPEX per Square Foot Growth Rate,Maintenance CapEx per Square Foot Growth Rate,D&A per Square Foot Growth Rate,Growth CapEx per New Square Foot Growth Rate,Membership and Other Income Growth Rate
str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""FY19""",1129,510329,4076,21957,-10344,10678,452.018601,432.570416,8.932642,9.457927,514405,-6963.212488,14993.787512,null,null,null,null,null,null,null
"""FY20""",1129,519926,4038,20568,-10705,10987,460.519043,442.301151,9.481842,9.731621,523964,-6522.719609,14045.280391,null,null,null,null,null,null,null
"""FY21""",1121,555233,3918,22548,-10604,11152,495.301517,475.187333,9.392383,9.94826,559151,-7150.636024,15397.363976,null,null,null,null,null,null,null
"""FY22""",null,null,null,null,null,null,null,null,null,null,null,null,null,0.02,0.03,0.03,0.03,0.025,NaN,0.03
"""FY23""",null,null,null,null,null,null,null,null,null,null,null,null,null,0.02,0.03,0.03,0.03,0.025,0.03,0.03
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""FY27""",null,null,null,null,null,null,null,null,null,null,null,null,null,0.01,0.02,0.02,0.02,0.015,0.02,0.02
"""FY28""",null,null,null,null,null,null,null,null,null,null,null,null,null,0.005,0.015,0.015,0.015,0.01,0.02,0.015
"""FY29""",null,null,null,null,null,null,null,null,null,null,null,null,null,0.005,0.015,0.015,0.015,0.01,0.015,0.015


In [48]:
joined_df.filter(pl.col("Year") == "FY25")

Year,Retail Square Foot,Net Sales,Membership & Other Income,Operating Income (EBIT),Capital Expenditures,Depreciation & Amortization,Sales per Square Foot,COGS and OpEx per Square Foot,Maintenance CapEx per Square Foot,D&A per Square Foot,Total Revenue,"(-) Taxes, Excluding Effect of Interest:",Net Operating Profit After Tax (NOPAT),Retail Square Foot Growth Rate,Sales per Square Foot Growth Rate,COGS and OPEX per Square Foot Growth Rate,Maintenance CapEx per Square Foot Growth Rate,D&A per Square Foot Growth Rate,Growth CapEx per New Square Foot Growth Rate,Membership and Other Income Growth Rate
str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""FY25""",null,null,null,null,null,null,null,null,null,null,null,null,null,0.015,0.025,0.025,0.025,0.02,0.025,0.025


In [52]:
"""
syntax I want:

"x".prev(1) * "r"
"""

'\nsyntax I want:\n\n"x".prev(1) * "r"\n'